In [1]:
using Revise
using Test
using SymbolicUtils

includet("main_whit.jl")
includet("main_symb.jl")


In [2]:
function g(i, d)
  if d == 1
    (y[i] - y[i-d]) / (x[i] - x[i-d])
  elseif d > 1
    return (g(i, d - 1) - g(i - 1, d - 1)) / (x[i] - x[i-d]) #* d
  end
end

n = 6
p = 3
x = Vec(:x, n)
y = Vec(:y, n);

# @variables i
# y[i]

6-element Vector{Num}:
 y₁
 y₂
 y₃
 y₄
 y₅
 y₆

In [3]:
ddmat_full(x, 2)

4×6 Matrix{Num}:
 (-(-1 / (-x₁ + x₂))) / (-x₁ + x₃)  …                            0
                                 0                               0
                                 0                               0
                                 0     1 / ((-x₄ + x₆)*(-x₅ + x₆))

In [5]:
d = 3
Matrix(ddmat(x, d)) - ddmat_full(x, d)

3×6 Matrix{Num}:
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0

In [42]:
d = 4
Matrix(ddmat_band(x, d)) - ddmat_full(x, d)

2×6 Matrix{Num}:
 0  0  0  0  0  0
 0  0  0  0  0  0

In [17]:
a = Mat(:a, 7)
force_band!(a, 2, 3)
force_upper!(a)
a

7×7 Matrix{Num}:
 a₁ˏ₁  a₁ˏ₂  a₁ˏ₃  a₁ˏ₄     0     0     0
    0  a₂ˏ₂  a₂ˏ₃  a₂ˏ₄  a₂ˏ₅     0     0
    0     0  a₃ˏ₃  a₃ˏ₄  a₃ˏ₅  a₃ˏ₆     0
    0     0     0  a₄ˏ₄  a₄ˏ₅  a₄ˏ₆  a₄ˏ₇
    0     0     0     0  a₅ˏ₅  a₅ˏ₆  a₅ˏ₇
    0     0     0     0     0  a₆ˏ₆  a₆ˏ₇
    0     0     0     0     0     0  a₇ˏ₇

In [18]:
U = BandedMat(a, 0, 3; zipped=false)
# D = ddmat_band(x, d)
# 上三角
# D' * D

7×4 Matrix{Num}:
 a₁ˏ₁  a₁ˏ₂  a₁ˏ₃  a₁ˏ₄
 a₂ˏ₂  a₂ˏ₃  a₂ˏ₄  a₂ˏ₅
 a₃ˏ₃  a₃ˏ₄  a₃ˏ₅  a₃ˏ₆
 a₄ˏ₄  a₄ˏ₅  a₄ˏ₆  a₄ˏ₇
 a₅ˏ₅  a₅ˏ₆  a₅ˏ₇     0
 a₆ˏ₆  a₆ˏ₇     0     0
 a₇ˏ₇     0     0     0

BandedMat{Num}, size=(7, 7), bandwidth=(0, 3), zipped=true


In [33]:
r = force_lower!(a' * a)

7×7 Matrix{Num}:
    a₁ˏ₁^2                      0  …                                  0
 a₁ˏ₁*a₁ˏ₂        a₁ˏ₂^2 + a₂ˏ₂^2                                     0
 a₁ˏ₁*a₁ˏ₃  a₁ˏ₂*a₁ˏ₃ + a₂ˏ₂*a₂ˏ₃                                     0
 a₁ˏ₁*a₁ˏ₄  a₁ˏ₂*a₁ˏ₄ + a₂ˏ₂*a₂ˏ₄                                     0
         0              a₂ˏ₂*a₂ˏ₅                                     0
         0                      0  …                                  0
         0                      0     a₄ˏ₇^2 + a₅ˏ₇^2 + a₆ˏ₇^2 + a₇ˏ₇^2

In [41]:
r2 = U_sq(U)
Matrix(r2) - r


7×7 Matrix{Num}:
 0  0  0  0  0  0  0
 0  0  0  0  0  0  0
 0  0  0  0  0  0  0
 0  0  0  0  0  0  0
 0  0  0  0  0  0  0
 0  0  0  0  0  0  0
 0  0  0  0  0  0  0

In [96]:
g.(p+1:n, p) # 需要进一步将y分解为三项


3-element Vector{Num}:
 (((-y₃ + y₄) / (-x₃ + x₄) + (y₂ - y₃) / (-x₂ + x₃)) / (-x₂ + x₄) + ((-y₁ + y₂) / (-x₁ + x₂) + (y₂ - y₃) / (-x₂ + x₃)) / (-x₁ + x₃)) / (-x₁ + x₄)
 (((-y₂ + y₃) / (-x₂ + x₃) + (y₃ - y₄) / (-x₃ + x₄)) / (-x₂ + x₄) + ((y₃ - y₄) / (-x₃ + x₄) + (-y₄ + y₅) / (-x₄ + x₅)) / (-x₃ + x₅)) / (-x₂ + x₅)
 (((-y₅ + y₆) / (-x₅ + x₆) + (y₄ - y₅) / (-x₄ + x₅)) / (-x₄ + x₆) + ((-y₃ + y₄) / (-x₃ + x₄) + (y₄ - y₅) / (-x₄ + x₅)) / (-x₃ + x₅)) / (-x₃ + x₆)

In [93]:
a = Mat(:a, n) ## 每一行均乘
diagm(x) * a

6×6 Matrix{Num}:
 a₁ˏ₁*x₁  a₁ˏ₂*x₁  a₁ˏ₃*x₁  a₁ˏ₄*x₁  a₁ˏ₅*x₁  a₁ˏ₆*x₁
 a₂ˏ₁*x₂  a₂ˏ₂*x₂  a₂ˏ₃*x₂  a₂ˏ₄*x₂  a₂ˏ₅*x₂  a₂ˏ₆*x₂
 a₃ˏ₁*x₃  a₃ˏ₂*x₃  a₃ˏ₃*x₃  a₃ˏ₄*x₃  a₃ˏ₅*x₃  a₃ˏ₆*x₃
 a₄ˏ₁*x₄  a₄ˏ₂*x₄  a₄ˏ₃*x₄  a₄ˏ₄*x₄  a₄ˏ₅*x₄  a₄ˏ₆*x₄
 a₅ˏ₁*x₅  a₅ˏ₂*x₅  a₅ˏ₃*x₅  a₅ˏ₄*x₅  a₅ˏ₅*x₅  a₅ˏ₆*x₅
 a₆ˏ₁*x₆  a₆ˏ₂*x₆  a₆ˏ₃*x₆  a₆ˏ₄*x₆  a₆ˏ₅*x₆  a₆ˏ₆*x₆

In [89]:
p = 2
D = ddmat(x, p) |> Matrix
simplify(D)
# r = BandedMat(D, 0, p; zipped=false)
BandedMat(D, 0, p; zipped=false)

4×3 Matrix{Num}:
 (-(-1 / (-x₁ + x₂))) / (-x₁ + x₃)  …  1 / ((-x₁ + x₃)*(-x₂ + x₃))
 (-(-1 / (-x₂ + x₃))) / (-x₂ + x₄)     1 / ((-x₂ + x₄)*(-x₃ + x₄))
 (-(-1 / (-x₃ + x₄))) / (-x₃ + x₅)     1 / ((-x₃ + x₅)*(-x₄ + x₅))
 (-(-1 / (-x₄ + x₅))) / (-x₄ + x₆)     1 / ((-x₄ + x₆)*(-x₅ + x₆))

BandedMat{Num}, size=(4, 6), bandwidth=(0, 2), zipped=true


3×4 Matrix{Num}:
 (-((-(-1 / (-x₁ + x₂))) / (-x₁ + x₃))) / (-x₁ + x₄)  …  1 / ((-x₁ + x₄)*(-x₂ + x₄)*(-x₃ + x₄))
 (-((-(-1 / (-x₂ + x₃))) / (-x₂ + x₄))) / (-x₂ + x₅)     1 / ((-x₂ + x₅)*(-x₃ + x₅)*(-x₄ + x₅))
 (-((-(-1 / (-x₃ + x₄))) / (-x₃ + x₅))) / (-x₃ + x₆)     1 / ((-x₃ + x₆)*(-x₄ + x₆)*(-x₅ + x₆))

BandedMat{Num}, size=(3, 6), bandwidth=(0, 3), zipped=true


In [10]:
BD = BandedMat(D, 0, 3; zipped=false)
BD'

MethodError: MethodError: no method matching size(::BandedMat{Num}, ::Int64)

Closest candidates are:
  size(!Matched::BitVector, ::Integer)
   @ Base bitarray.jl:107
  size(!Matched::Type{<:Diagonal{T, StaticArraysCore.SVector{N, T}} where T}, ::Int64) where N
   @ StaticArrays C:\Users\kongdd\.julia\packages\StaticArrays\EHHaF\src\SDiagonal.jl:18
  size(!Matched::Type{<:Union{Adjoint{T, <:Union{StaticArraysCore.StaticArray{Tuple{var"#s2"}, T, 1} where var"#s2", StaticArraysCore.StaticArray{Tuple{var"#s3", var"#s4"}, T, 2} where {var"#s3", var"#s4"}}}, Diagonal{T, <:StaticArraysCore.StaticArray{Tuple{var"#s14"}, T, 1} where var"#s14"}, Hermitian{T, <:StaticArraysCore.StaticArray{Tuple{var"#s11", var"#s12"}, T, 2} where {var"#s11", var"#s12"}}, LowerTriangular{T, <:StaticArraysCore.StaticArray{Tuple{var"#s19", var"#s20"}, T, 2} where {var"#s19", var"#s20"}}, Symmetric{T, <:StaticArraysCore.StaticArray{Tuple{var"#s8", var"#s9"}, T, 2} where {var"#s8", var"#s9"}}, Transpose{T, <:Union{St

In [9]:
# 转置出现了错误
BandedMat(D', 3, 0; zipped=false)

10×4 Matrix{Num}:
   0     0    0  -x₁
   0     0  3x₂  -x₂
   0  -3x₃  3x₃  -x₃
  x₄  -3x₄  3x₄  -x₄
  x₅  -3x₅  3x₅  -x₅
  x₆  -3x₆  3x₆  -x₆
  x₇  -3x₇  3x₇  -x₇
  x₈  -3x₈  3x₈    0
  x₉  -3x₉    0    0
 x₁₀     0    0    0

BandedMat{Num}, size=(10, 7), bandwidth=(3, 0)


In [51]:
## 设计一个带状矩阵的乘法
# D' * D



10×10 Matrix{Num}:
    x₁^2   -3x₁*x₂    3x₁*x₃    -x₁*x₄  …         0         0         0
 -3x₁*x₂  10(x₂^2)  -12x₂*x₃    6x₂*x₄            0         0         0
  3x₁*x₃  -12x₂*x₃  19(x₃^2)  -15x₃*x₄            0         0         0
  -x₁*x₄    6x₂*x₄  -15x₃*x₄  20(x₄^2)            0         0         0
       0    -x₂*x₅    6x₃*x₅  -15x₄*x₅       -x₅*x₈         0         0
       0         0    -x₃*x₆    6x₄*x₆  …    6x₆*x₈    -x₆*x₉         0
       0         0         0    -x₄*x₇     -15x₇*x₈    6x₇*x₉   -x₁₀*x₇
       0         0         0         0     19(x₈^2)  -12x₈*x₉   3x₁₀*x₈
       0         0         0         0     -12x₈*x₉  10(x₉^2)  -3x₁₀*x₉
       0         0         0         0      3x₁₀*x₈  -3x₁₀*x₉     x₁₀^2

In [37]:
lambda = 2.0
d = 2
n = 10
y = rand(n)
# x = 1:n
w = ones(n)

n = length(y)
# D = ddmat(speye(n), d)
D = diff(speye(n), 2)

#  1   1
#  1  -2   1
# -1   3  -3   1
#  1  -4   6  -4   1 
# -1   5  -10   10   -5    1

# W = SparseArrays.spdiagm(w)
# A = W + lambda * D' * D
# # z, cve = WHIT(y, w)


8×10 SparseMatrixCSC{Int64, Int64} with 24 stored entries:
 1  -2   1   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  ⋅
 ⋅   1  -2   1   ⋅   ⋅   ⋅   ⋅   ⋅  ⋅
 ⋅   ⋅   1  -2   1   ⋅   ⋅   ⋅   ⋅  ⋅
 ⋅   ⋅   ⋅   1  -2   1   ⋅   ⋅   ⋅  ⋅
 ⋅   ⋅   ⋅   ⋅   1  -2   1   ⋅   ⋅  ⋅
 ⋅   ⋅   ⋅   ⋅   ⋅   1  -2   1   ⋅  ⋅
 ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   1  -2   1  ⋅
 ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   1  -2  1

10×10 Matrix{Num}:
 x₁   0   0   0   0   0   0   0   0    0
  0  x₂   0   0   0   0   0   0   0    0
  0   0  x₃   0   0   0   0   0   0    0
  0   0   0  x₄   0   0   0   0   0    0
  0   0   0   0  x₅   0   0   0   0    0
  0   0   0   0   0  x₆   0   0   0    0
  0   0   0   0   0   0  x₇   0   0    0
  0   0   0   0   0   0   0  x₈   0    0
  0   0   0   0   0   0   0   0  x₉    0
  0   0   0   0   0   0   0   0   0  x₁₀

In [5]:
z2, cve2 = WHIT(y, w, x)
@test z2 ≈ z
@test cve2 ≈ cve


Test Passed